In [1]:
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"    
import tensorflow as tf
import sys
sys.path.append("..")
from Vocabulary import *
import math
import numpy as np

#only for nomalised a and b
def fast_cosine(a,b):
    return np.dot(a,b)


    

In [15]:
with open(r'..\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:
    vocab = Vocabulary()
    vocab.load('../vocabs/text8_filtered100')

    matrix_nr = vocab.get_id_text('man')
    print(matrix_nr)

    #hdf_file = h5py.File('testRunCleanSplitted300.hdf5', 'r')
    hdf_file = h5py.File('S:\\testRunCleanSplitted300.hdf5','r')
    weights = hdf_file.get('weights')
    weights = weights[:]#unpack hdf to numpy
    weights_normalized = tf.nn.l2_normalize(weights,axis = 0)
    context_weights = hdf_file.get('context-weights')

    cosine_loss = tf.keras.losses.CosineSimilarity(axis=0)

    losses = []
    scores = []
    print(weights_normalized.shape)
    print(vocab.get_size())
    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass
        print(line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try: 
            id1 = vocab.get_id_text(word1)
            id2 = vocab.get_id_text(word2)
            print(id1)
            print(id2)
            vector1 = weights[id1]
            vector2 = weights[id2]
            loss = np.dot(vector1,vector2)/(np.linalg.norm(vector1) * np.linalg.norm(vector2))
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))
    

191
(11815, 300)
11815
tiger	cat	7.35

4931
2642
0.2119302 0.735
tiger	tiger	10.00

4931
4931
1.0 1.0
plane	car	5.77

8044
5738
0.32366857 0.577
train	car	6.31

1884
5738
0.29396862 0.631
television	radio	6.77

3034
5721
0.55902517 0.6769999999999999
media	radio	7.42

2088
5721
0.3272376 0.742
bread	butter	6.19

562
10605
0.26793596 0.619
cucumber	potato	5.92

0 0.592
doctor	nurse	7.00

6485
4134
0.19778992 0.7
professor	doctor	6.62

1419
6485
0.26489064 0.662
student	professor	6.81

1890
1419
0.3754596 0.6809999999999999
smart	stupid	5.81

0 0.581
wood	forest	7.73

2998
2332
0.23730569 0.773
money	cash	9.15

2539
2531
0.30299532 0.915
king	queen	8.58

55
3989
0.45994514 0.858
king	rook	5.92

0 0.592
bishop	rabbi	6.69

5664
9444
0.19418396 0.669
fuck	sex	9.44

0 0.944
football	soccer	9.03

6953
7010
0.48357365 0.9029999999999999
football	basketball	6.81

6953
7700
0.36690888 0.6809999999999999
football	tennis	6.63

6953
6969
0.25196275 0.663
Arafat	Jackson	2.50

0 0.25
physics	chemistr

In [17]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.2558312251250799, pvalue=0.00022958150377171873)